 # Notebook description
Preparing the database for reporting purposes. Flight delay percentage reporting areas:
1. Displaying the TOP 10 (highest percentage of delays) airports based on the number of departures, displaying information about the number of arrivals - `top_airports_by_departure` view.  
  2. Displaying the TOP 10 (highest percentage of delays) flight routes. Assumption: the minimum number of flights on the route is at least 10,000 flights - `top_reliability_roads` view.  
  3. Comparison of 2019 vs. 2020 to track the impact of COVID on flight operations:
      - month to month - `year_to_year_comparison` view,  
      - day to day - `day_to_day_comparison` view.  
  4. Daily aircraft flight delay rate.

 # Technical approach to the problem
The goal is to separate the data preparation layer (report logic) from the presentation layer (visualization).
  The presentation layer (chart/report) does not implement business logic to process data.

# Preparing the database
Creating a schema - `reporting`.  
Creating views with report areas.

In [1]:
import psycopg2
from psycopg2 import connect
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
username = 'DB_ID'
password = 'DB_PASS'
host = 'localhost'
database = 'airlines'
port = 5432

In [5]:
con = psycopg2.connect(
        dbname=database,
        user=username,
        password=password,
        host=host,
        port=port
    )
cursor = con.cursor()
print("The connection to the database has been established.")

con.rollback()

The connection to the database has been established.


In [11]:
# Loading the reporting.sql file
file_path = './reporting.sql'

with open(file_path, 'r') as file:
    sql_reporting = file.read()

In [12]:
# Dividing the script into individual queries
queries = sql_reporting.split(';')
queries = [query.strip() + ';' for query in queries if query.strip()]

In [13]:
# Execution of individual queries
for query in queries:
    try:
        cursor.execute(query)
        con.commit()  
        print("A query has been performed:", query)
    except psycopg2.Error as e:
        print("Error executing the query:", query)
        print("Error message:", e)
        con.rollback()  

A query has been performed: DROP SCHEMA IF EXISTS reporting CASCADE;
A query has been performed: CREATE SCHEMA reporting;
A query has been performed: CREATE OR REPLACE VIEW reporting.flight AS
SELECT 
    *,
    CASE 
        WHEN dep_delay_new > 0 THEN 1
        ELSE 0 
    END AS is_delayed
FROM
    public.flight
WHERE
    cancelled = 0;
A query has been performed: CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT
    rf.origin_airport_id,
    al1.name AS origin_airport_name,
    rf.dest_airport_id,
    al2.name AS dest_airport_name,
    rf.year,
    COUNT(*) AS cnt,
    ROUND(AVG(rf.is_delayed)::numeric, 2) AS reliability,
    RANK() OVER (ORDER BY ROUND(AVG(rf.is_delayed)::numeric, 2) DESC) AS nb
FROM
    reporting.flight rf
JOIN
    airport_list al1 ON rf.origin_airport_id = al1.origin_airport_id
JOIN
    airport_list al2 ON rf.dest_airport_id = al2.origin_airport_id
GROUP BY
    rf.origin_airport_id,
    al1.name,
    rf.dest_airport_id,
    al2.name,
    rf.year
H

In [14]:
# Approval of all operations on the database, i.e. creation of views
con.commit()

In [15]:
# checking if views exist
def check_if_table_exists(table_name):
    msg = f"I check if the table exists {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    cursor.execute(query)
    print('OK!')

In [16]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [17]:
for view in views_to_test:
    check_if_table_exists(view)

I check if the table exists reporting.flight
OK!
I check if the table exists reporting.top_reliability_roads
OK!
I check if the table exists reporting.year_to_year_comparision
OK!
I check if the table exists reporting.day_to_day_comparision
OK!
I check if the table exists reporting.day_by_day_reliability
OK!


In [18]:
con.close()